In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../../")

from experiment import Exp

from dataloaders import data_set,data_dict
import torch
import yaml
import os

In [2]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict()   

args.to_save_path     = r"E:\TECO_Works\Conference\ISWC2022\Run_logs"              
args.freq_save_path   = r"E:\TECO_Works\Conference\ISWC2022\Freq_data"
args.window_save_path = r"E:\TECO_Works\Conference\ISWC2022\Sliding_window"
args.root_path        = r"E:\datasets"


args.drop_transition  = False
args.datanorm_type    = "standardization" # None ,"standardization", "minmax"


args.batch_size       = 128                                                     
args.shuffle          = True
args.drop_last        = False
args.train_vali_quote = 0.90                                           


# training setting 
args.train_epochs            = 150

args.learning_rate           = 0.001  
args.learning_rate_patience  = 5
args.learning_rate_factor    = 0.1


args.early_stop_patience     = 15

args.use_gpu                 = False if torch.cuda.is_available() else False
args.gpu                     = 0
args.use_multi_gpu           = False

args.optimizer               = "Adam"
args.criterion               = "CrossEntropy"

In [3]:
args.seed                             = 3


args.data_name                        =  "hapt"
window_seconds                        = 2.56
args.wavelet_filtering                = True
args.wavelet_filtering_regularization = True
args.regulatization_tradeoff          = 0.0001
args.number_wavelet_filtering = 10


args.difference       = False 
args.filtering        =  False
args.magnitude        =  False
args.weighted_sampler = False




args.pos_select       = None
args.sensor_select    = None


args.representation_type = "time"
args.exp_mode            = "LOCV"

config_file = open('../../configs/data.yaml', mode='r')
data_config = yaml.load(config_file, Loader=yaml.FullLoader)
config = data_config[args.data_name]

args.root_path       = os.path.join(args.root_path,config["filename"])
args.sampling_freq   = config["sampling_freq"]
args.num_classes     =  config["num_classes"]
args.windowsize      =   int(window_seconds * args.sampling_freq) 
args.input_length    =  args.windowsize
# input information
args.c_in            = config["num_channels"]

if args.wavelet_filtering :
    
    if args.windowsize%2==1:
        N_ds = int(torch.log2(torch.tensor(args.windowsize-1)).floor()) - 2
    else:
        N_ds = int(torch.log2(torch.tensor(args.windowsize)).floor()) - 2

    args.f_in            =  args.number_wavelet_filtering*N_ds+1
else:
    args.f_in            =  1



In [4]:
args.filter_scaling_factor = 1
args.model_type              = "deepconvlstm"

In [5]:
exp = Exp(args)

Use CPU
Wavelet Filtering Regularization
Build the DeepConvLSTM model!
Done!
Parameter : 5865727
Set the seed as :  3


In [26]:
args.wavelet_filtering_finetuning = True
args.percent = 0.5
total = args.f_in


#gamma_weight = exp.model.gamma.squeeze().abs().clone()

gamma_weight = torch.rand(total)
y, i = torch.sort(gamma_weight)
thre_index = int(total * args.percent)
thre = y[thre_index]
print(thre)


mask = gamma_weight.data.gt(thre).float().cuda()

gamma_weight = 

tensor(0.5356)


In [25]:
gamma_weight

tensor([0.0043, 0.1056, 0.2858, 0.0270, 0.4716, 0.0601, 0.7719, 0.7437, 0.5944,
        0.8879, 0.4510, 0.7995, 0.1498, 0.4015, 0.0542, 0.4594, 0.1756, 0.9492,
        0.8473, 0.8749, 0.6483, 0.2148, 0.9493, 0.0121, 0.1809, 0.1877, 0.2927,
        0.5997, 0.1213, 0.1148, 0.2609, 0.8052, 0.1394, 0.0516, 0.0666, 0.2540,
        0.3686, 0.0043, 0.2659, 0.5096, 0.5824, 0.4283, 0.4433, 0.1747, 0.7562,
        0.3795, 0.1581, 0.3796, 0.4301, 0.9623, 0.7606])

In [24]:
gamma_weight.data

tensor([0.0043, 0.1056, 0.2858, 0.0270, 0.4716, 0.0601, 0.7719, 0.7437, 0.5944,
        0.8879, 0.4510, 0.7995, 0.1498, 0.4015, 0.0542, 0.4594, 0.1756, 0.9492,
        0.8473, 0.8749, 0.6483, 0.2148, 0.9493, 0.0121, 0.1809, 0.1877, 0.2927,
        0.5997, 0.1213, 0.1148, 0.2609, 0.8052, 0.1394, 0.0516, 0.0666, 0.2540,
        0.3686, 0.0043, 0.2659, 0.5096, 0.5824, 0.4283, 0.4433, 0.1747, 0.7562,
        0.3795, 0.1581, 0.3796, 0.4301, 0.9623, 0.7606])

In [14]:
total = exp.model.gamma.squeeze().shape[0]
y, i = torch.sort(exp.model.gamma.squeeze())

In [15]:
y

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=torch.float64, grad_fn=<SortBackward0>)

In [16]:
i

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])